In [14]:
import torch
import torch.nn as nn
import numpy as np
import sys
import os
import random
import matplotlib.pyplot as plt

src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)
    
from utils import MIMONetDataset, DeepONetDataset, ChannelScaler
from mimonet import MIMONet

In [15]:
# check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [16]:
# set working directory
working_dir = "/projects/bcnx/kazumak2/MIMONet/Subchannel/"
data_dir = os.path.join(working_dir, "data")

## load datasets

### Load sharing parameters/dataset

In [17]:
# trunk dataset
trunk_input = np.load(os.path.join(data_dir, "share/trunk_input.npz"))['trunk']

### Training data

In [18]:
# training data
train_branch = np.load(os.path.join(data_dir, "training/train_branch_input.npz"))
train_branch_1 = train_branch['func_params']
train_branch_2 = train_branch['stat_params']

# [samples, channel, gridpoints]
train_target = np.load(os.path.join(data_dir, "training/train_target.npz"))['target']
# convert to [samples, gridpoints, channel]
train_target = np.moveaxis(train_target, 1, 2)

print("train_branch_1 shape:", train_branch_1.shape)
print("train_branch_2 shape:", train_branch_2.shape)
print("train_target shape:", train_target.shape)

train_branch_1 shape: (4000, 100)
train_branch_2 shape: (4000, 2)
train_target shape: (4000, 1733, 3)


In [19]:
# scaling the functional input data using predefined mean and std
f_mean = np.load(os.path.join(data_dir, "share/func_mean_std_params.npz"))['mean']
f_std = np.load(os.path.join(data_dir, "share/func_mean_std_params.npz"))['std']

train_branch_1 = (train_branch_1 - f_mean) / f_std

# scaling the static input data using predefined mean and std
s_mean = np.load(os.path.join(data_dir, "share/stat_mean_std_params.npz"))['mean']
s_std = np.load(os.path.join(data_dir, "share/stat_mean_std_params.npz"))['std']

for i in range(s_mean.shape[0]):
    train_branch_2[:, i] = (train_branch_2[:, i] - s_mean[i]) / s_std[i]

### Test data

In [20]:
test_branch = np.load(os.path.join(data_dir, "test/test_branch_input.npz"))
test_branch_1 = test_branch['func_params']
test_branch_2 = test_branch['stat_params']

test_target = np.load(os.path.join(data_dir, "test/test_target.npz"))['target']
test_target = np.moveaxis(test_target, 1, 2)

print("test_branch_1 shape:", test_branch_1.shape)
print("test_branch_2 shape:", test_branch_2.shape)
print("test_target shape:", test_target.shape)

# scaling the functional input data using predefined mean and std
test_branch_1 = (test_branch_1 - f_mean) / f_std
# scaling the static input data using predefined mean and std
for i in range(s_mean.shape[0]):
    test_branch_2[:, i] = (test_branch_2[:, i] - s_mean[i]) / s_std[i]

test_branch_1 shape: (1000, 100)
test_branch_2 shape: (1000, 2)
test_target shape: (1000, 1733, 3)


### Scaling the target data

In [21]:
# scaling the target data
'''  
note: reverse the scaling for the target data
train_target = scaler.inverse_transform(train_target_scaled)
test_target = scaler.inverse_transform(test_target_scaled)
'''
scaler = ChannelScaler(method='minmax', feature_range=(-1, 1))
scaler.fit(train_target)
train_target_scaled = scaler.transform(train_target)
test_target_scaled = scaler.transform(test_target)


## Torch Dataset and DataLoader

In [22]:
# test dataset and dataloader
test_dataset = MIMONetDataset(
    [test_branch_1, test_branch_2],  # branch_data_list
    trunk_input,                     # trunk_data
    test_target_scaled               # target_data
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # set to 1 for testing
    shuffle=False,
    num_workers=0
)

In [23]:
train_dataset = MIMONetDataset(
    [train_branch_1, train_branch_2],  # branch_data_list
    trunk_input,                       # trunk_data
    train_target_scaled                # target_data
)

## MIMONet Model

In [24]:
# Architecture parameters
dim = 256
branch_input_dim1 = 100
branch_input_dim2 = 2
trunk_input_dim = 2

# Define MIONet instance (no Fourier, no final linear)
model = MIMONet(
    branch_arch_list=[
        [branch_input_dim1, 512, 512, 512, dim],
        [branch_input_dim2, 512, 512, 512, dim]
    ],
    trunk_arch=[trunk_input_dim, 256, 256, dim],
    num_outputs=3, 
    activation_fn=nn.ReLU,
    merge_type='mul'  # or 'sum'
)

model = model.to(device)

# Print parameter count
num_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {num_params:,}")

Total number of parameters: 1,630,467


In [25]:
from scripts.training import train_model

In [ ]:
train_model(
    model=model,
    dataset=train_dataset,
    device='cuda',
    num_epochs=500,
    batch_size=4,
    lr=1e-3,
    patience=10,
    multi_gpu=False,
    working_dir="Subchannel"   # ✅ This creates Subchannel/checkpoints/
)

print("Training completed.")

Epoch 1, Train Loss: 0.194447, Val Loss: 0.191655
Epoch 2, Train Loss: 0.178851, Val Loss: 0.138314
Epoch 3, Train Loss: 0.128785, Val Loss: 0.121873
Epoch 4, Train Loss: 0.114465, Val Loss: 0.110734
Epoch 5, Train Loss: 0.110145, Val Loss: 0.128395
Epoch 6, Train Loss: 0.108399, Val Loss: 0.137311
Epoch 7, Train Loss: 0.107725, Val Loss: 0.110215
Epoch 8, Train Loss: 0.106625, Val Loss: 0.108952
Epoch 9, Train Loss: 0.107875, Val Loss: 0.110263
Epoch 10, Train Loss: 0.105742, Val Loss: 0.127255
Epoch 11, Train Loss: 0.105762, Val Loss: 0.107062
Epoch 12, Train Loss: 0.104573, Val Loss: 0.106647
Epoch 13, Train Loss: 0.103852, Val Loss: 0.104907
Epoch 14, Train Loss: 0.102916, Val Loss: 0.107248
Epoch 15, Train Loss: 0.101873, Val Loss: 0.103045
Epoch 16, Train Loss: 0.100697, Val Loss: 0.105633
Epoch 17, Train Loss: 0.098298, Val Loss: 0.096885
Epoch 18, Train Loss: 0.095184, Val Loss: 0.096911
Epoch 19, Train Loss: 0.092586, Val Loss: 0.096649
Epoch 20, Train Loss: 0.091396, Val Loss

KeyboardInterrupt: 